In [1]:
#pip install builtwith

In [2]:
#pip install selenium

In [3]:
import builtwith
builtwith.parse('https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-23/1')

{'javascript-frameworks': ['AngularJS', 'Modernizr', 'jQuery'],
 'javascript-graphics': ['D3', 'Highcharts'],
 'font-scripts': ['Google Font API'],
 'web-frameworks': ['Twitter Bootstrap']}

In [4]:
import requests

# Petición a la página web
#url principal
#TODO Navegar desde la pagina principal al dia de hoy
#url = "https://demanda.ree.es/visiona/peninsula/demanda/total"
url_1 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-31/1"
url_2 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-31/2"
url_3 = "https://demanda.ree.es/visiona/peninsula/demanda/tablas/2021-10-31/3"
page_1 = requests.get(url_1)
page_2 = requests.get(url_2)
page_3 = requests.get(url_3)

# Lectura del html del response
from bs4 import BeautifulSoup
soup_1 = BeautifulSoup(page_1.content)
soup_2 = BeautifulSoup(page_2.content)
soup_3 = BeautifulSoup(page_3.content)

C:\Users\Manuel\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd

#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = '/users/Manuel/Downloads/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url_1)

#Inicializamos los arrays para almacenar las variables
dias = list()
horas = list()
lectura_real = list()
lectura_prevista = list()
lectura_programada = list()
lectura_eolica = list()

#Navegamos por los 30 dias anteriores
for i in range(2):            
    print("hoja",i)  
    # Leemos la información por horas    
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Dia Hora
            xpath_diahora = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[1]"
            element_diahora = driver.find_element_by_xpath(xpath_diahora)
            dia = element_diahora.text.split()[0]
            hora = element_diahora.text.split()[1]
            dias.append(dia)
            horas.append(hora)

            #Real
            xpath_real = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[2]"
            element_real = driver.find_element_by_xpath(xpath_real)
            lectura_real.append(element_real.text)

            #Prevista
            xpath_prevista = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[3]"
            element_prevista = driver.find_element_by_xpath(xpath_prevista)
            lectura_prevista.append(element_prevista.text)

            #Programada
            xpath_programada = "/html/body/div/md-content/div/section/div[2]/div[2]/table[1]/tbody/tr["+ str(t) +"]/td[4]"
            element_programada = driver.find_element_by_xpath(xpath_programada)
            lectura_programada.append(element_programada.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            dias.append("")
            horas.append("")
            lectura_real.append("")
            lectura_prevista.append("")
            lectura_programada.append("")

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_1 = pd.DataFrame({'Fecha': dias, 'Hora': horas, 'Lectura Real': lectura_real, 'Lectura_prevista': lectura_prevista,\
                     'Lectura_programada': lectura_programada})

<ipython-input-5-f4856d74f257>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-5-f4856d74f257>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_diahora = driver.find_element_by_xpath(xpath_diahora)
<ipython-input-5-f4856d74f257>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_real = driver.find_element_by_xpath(xpath_real)
<ipython-input-5-f4856d74f257>:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_prevista = driver.find_element_by_xpath(xpath_prevista)
<ipython-input-5-f4856d74f257>:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_programada = driver.find_element_by_xpath(xpath_programada)


hoja 1


In [6]:
#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = '/users/Manuel/Downloads/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url_2)

#Inicializamos los arrays para almacenar las variables
lectura_eolica = list()
lectura_nuclear = list()
lectura_fuel_gas = list()
lectura_carbon = list()
lectura_ciclo_combinado = list()
lectura_hidraulica = list()
lectura_intercambios = list()
lectura_enlace_balear = list()
lectura_solar_fotovoltaica = list()
lectura_solar_termica = list()
lectura_termica_renovable = list()
lectura_cogeneracion_residuos = list()

#Navegamos por los 30 dias anteriores
for i in range(2):            
    print("hoja",i)  
    # Leemos la información por horas    
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Eólica
            xpath_eolica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[2]"
            element_eolica = driver.find_element_by_xpath(xpath_eolica)
            lectura_eolica.append(element_eolica.text)

            #Nuclear
            xpath_nuclear = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[3]"
            element_nuclear = driver.find_element_by_xpath(xpath_nuclear)
            lectura_nuclear.append(element_nuclear.text)

            #Fuel/Gas
            xpath_fuel_gas = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[4]"
            element_fuel_gas = driver.find_element_by_xpath(xpath_fuel_gas)
            lectura_fuel_gas.append(element_fuel_gas.text)

            #Carbon
            xpath_carbon = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[5]"
            element_carbon = driver.find_element_by_xpath(xpath_carbon)
            lectura_carbon.append(element_carbon.text)

            #Ciclo combinado
            xpath_ciclo_combinado = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[6]"
            element_ciclo_combinado = driver.find_element_by_xpath(xpath_ciclo_combinado)
            lectura_ciclo_combinado.append(element_ciclo_combinado.text)

            #Hidraulica
            xpath_hidraulica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[7]"
            element_hidraulica = driver.find_element_by_xpath(xpath_hidraulica)
            lectura_hidraulica.append(element_hidraulica.text)

            #Intercambios
            xpath_intercambios = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[8]"
            element_intercambios = driver.find_element_by_xpath(xpath_intercambios)
            lectura_intercambios.append(element_intercambios.text)

            #Enlace Balear
            xpath_enlace_balear = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[9]"
            element_enlace_balear = driver.find_element_by_xpath(xpath_enlace_balear)
            lectura_enlace_balear.append(element_enlace_balear.text)

            #Solar Fotovoltaica
            xpath_solar_fotovoltaica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[10]"
            element_solar_fotovoltaica = driver.find_element_by_xpath(xpath_solar_fotovoltaica)
            lectura_solar_fotovoltaica.append(element_solar_fotovoltaica.text)

            #Solar Termica
            xpath_solar_termica = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[11]"
            element_solar_termica = driver.find_element_by_xpath(xpath_solar_termica)
            lectura_solar_termica.append(element_solar_termica.text)

            #Termica Renovable
            xpath_termica_renovable = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[12]"
            element_termica_renovable = driver.find_element_by_xpath(xpath_termica_renovable)
            lectura_termica_renovable.append(element_termica_renovable.text)

            #Cogeneracion Residuos
            xpath_cogeneracion_residuos = "/html/body/div/md-content/div/section/div[2]/div[2]/table[2]/tbody/tr["+ str(t) +"]/td[13]"
            element_cogeneracion_residuos = driver.find_element_by_xpath(xpath_cogeneracion_residuos)
            lectura_cogeneracion_residuos.append(element_cogeneracion_residuos.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            lectura_eolica.append("")
            lectura_nuclear.append("")
            lectura_fuel_gas.append("")
            lectura_carbon.append("")
            lectura_ciclo_combinado.append("")
            lectura_hidraulica.append("")
            lectura_intercambios.append("")
            lectura_enlace_balear.append("")
            lectura_solar_fotovoltaica.append("")
            lectura_solar_termica.append("")
            lectura_termica_renovable.append("")
            lectura_cogeneracion_residuos.append("")
            

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_2 = pd.DataFrame({'Eolica' : lectura_eolica, 'Nuclear' : lectura_nuclear, 'Fuel/Gas' : lectura_fuel_gas,\
                     'Carbon' : lectura_carbon, 'Ciclo Combinado' : lectura_ciclo_combinado, 'Hidraulica' : lectura_hidraulica,\
                     'Intercambios' : lectura_intercambios, 'Enlace Balear' : lectura_enlace_balear,\
                     'Solar Fotovoltaica' : lectura_solar_fotovoltaica, 'Solar Termica' : lectura_solar_termica,\
                     'Termica Renovable' : lectura_termica_renovable, 'Cogeneracion y Residuos' : lectura_cogeneracion_residuos})

<ipython-input-6-1040f9435cc7>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-6-1040f9435cc7>:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_eolica = driver.find_element_by_xpath(xpath_eolica)
<ipython-input-6-1040f9435cc7>:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_nuclear = driver.find_element_by_xpath(xpath_nuclear)
<ipython-input-6-1040f9435cc7>:46: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_fuel_gas = driver.find_element_by_xpath(xpath_fuel_gas)
<ipython-input-6-1040f9435cc7>:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_carbon = driver.find_element_by_xpath(xpath_carbon)
<ipython-input-6-1040f9435cc7>:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_ciclo_combinado = driver.find_element_by_xpath(xpath_ciclo_combinado)
<ipyth

hoja 1


In [7]:
#opciones de navegacion
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = '/users/Manuel/Downloads/chromedriver'

#Abrimos el navegador
driver = webdriver.Chrome(driver_path)

#Navegamos a la url
driver.get(url_3)

#Inicializamos los arrays para almacenar las variables
lectura_emisiones_carbon = list()
lectura_emisiones_ciclo_combinado = list()
lectura_emisiones_cogeneracion_residuos = list()

#Navegamos por los 30 dias anteriores
for i in range(2):            
    print("hoja",i)  
    # Leemos la información por horas
    for t in range(20,164):
        # Escribimos este try para que en caso de no poder leer los datos de un día no se pare la lectura
        try:
            #Emisiones Carbon
            xpath_emisiones_carbon = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[5]"
            element_emisiones_carbon = driver.find_element_by_xpath(xpath_emisiones_carbon)
            lectura_emisiones_carbon.append(element_emisiones_carbon.text)

            #Emisiones Ciclo Combinado
            xpath_emisiones_ciclo_combinado = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[6]"
            element_emisiones_ciclo_combinado = driver.find_element_by_xpath(xpath_emisiones_ciclo_combinado)
            lectura_emisiones_ciclo_combinado.append(element_emisiones_ciclo_combinado.text)

            #Emisiones Cogeneracion y Residuos
            xpath_emisiones_cogeneracion_residuos = "/html/body/div/md-content/div/section/div[2]/div[2]/table[3]/tbody/tr["+ str(t) +"]/td[13]"
            element_emisiones_cogeneracion_residuos = driver.find_element_by_xpath(xpath_emisiones_cogeneracion_residuos)
            lectura_emisiones_cogeneracion_residuos.append(element_emisiones_cogeneracion_residuos.text)
        
        except:
            #Para evitar que los datos que no se pueden leer descuadren en el dataset se ponen vacios
            lectura_emisiones_carbon.append("")
            lectura_emisiones_ciclo_combinado.append("")
            lectura_emisiones_cogeneracion_residuos.append("")

    # Pulsamos click para navegar al dia anterior
    wait = WebDriverWait(driver, 5)
    elementButton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.md-icon-button main-menu md-button md-ink-ripple'.replace(' ','.'))))
    elementButton.click()


#Construimos el dataframe
df_3 = pd.DataFrame({'Emisiones Carbon': lectura_emisiones_carbon, 'Emisiones Ciclo Combinado': lectura_emisiones_ciclo_combinado,\
                     'Emisiones Cogeneracion y Residuos': lectura_emisiones_cogeneracion_residuos})

<ipython-input-7-b57c9a6ed435>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


hoja 0


<ipython-input-7-b57c9a6ed435>:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_carbon = driver.find_element_by_xpath(xpath_emisiones_carbon)
<ipython-input-7-b57c9a6ed435>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_ciclo_combinado = driver.find_element_by_xpath(xpath_emisiones_ciclo_combinado)
<ipython-input-7-b57c9a6ed435>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_emisiones_cogeneracion_residuos = driver.find_element_by_xpath(xpath_emisiones_cogeneracion_residuos)


hoja 1


In [12]:
dataset_final = pd.concat([df_1, df_2, df_3], axis=1)
dataset_final.to_csv('dataset_final.csv')
dataset_final.head(10)

,Fecha,Hora,Lectura Real,Lectura_prevista,Lectura_programada,Eolica,Nuclear,Fuel/Gas,Carbon,Ciclo Combinado,Hidraulica,Intercambios,Enlace Balear,Solar Fotovoltaica,Solar Termica,Termica Renovable,Cogeneracion y Residuos,Emisiones Carbon,Emisiones Ciclo Combinado,Emisiones Cogeneracion y Residuos
0,2021-10-31,00:00,23.031,23.041,22.324,9.293,5.947,,954,2.154,1.501,-492,-49,4,10,632,2.670,906,797,748
1,2021-10-31,00:10,22.890,22.875,22.324,9.406,5.939,,901,2.250,1.184,-429,-50,4,10,617,2.671,856,833,748
2,2021-10-31,00:20,22.561,22.693,22.324,9.478,5.942,,850,2.267,972,-588,-50,4,9,617,2.659,808,839,745
3,2021-10-31,00:30,22.498,22.493,22.324,9.562,5.945,,809,2.222,995,-687,-50,4,10,617,2.658,769,822,744
4,2021-10-31,00:40,22.168,22.277,22.324,9.791,5.942,,772,1.822,915,-715,-49,4,10,617,2.667,733,674,747
5,2021-10-31,00:50,22.134,22.043,22.324,10.062,5.945,,633,1.478,691,-313,-50,4,10,617,2.640,601,547,739
6,2021-10-31,01:00,21.752,21.792,20.935,10.058,5.945,,541,1.463,358,-211,-50,4,10,617,2.628,514,541,736
7,2021-10-31,01:10,21.382,21.534,20.935,10.061,5.935,,524,1.894,-141,-320,-50,4,10,616,2.551,498,701,714
8,2021-10-31,01:20,21.548,21.278,20.935,10.047,5.943,,515,2.220,-755,108,-50,4,9,616,2.545,489,821,713
9,2021-10-31,01:30,21.327,21.023,20.935,9.974,5.937,,495,2.311,-599,-176,-49,4,9,616,2.547,470,855,713
